In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Importing Necessary Library

In [ ]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import textwrap

%matplotlib inline
plt.rcParams['figure.figsize']=10,6
plt.rcParams['axes.grid']=True
plt.gray()

### Loading the Data 

In [ ]:
des = pd.read_csv('/kaggle/input/chai-time-data-science/Description.csv')
epsd = pd.read_csv('/kaggle/input/chai-time-data-science/Episodes.csv')
ytb = pd.read_csv('/kaggle/input/chai-time-data-science/YouTube Thumbnail Types.csv')
anchor = pd.read_csv('/kaggle/input/chai-time-data-science/Anchor Thumbnail Types.csv')

print('Description_shape...'+str(des.shape))
print('Episode_shape...'+str(epsd.shape))
print('Youtube_shape...'+str(ytb.shape))
print('Thumbail_shape...'+str(anchor.shape))

### Checking the Description and Episode tabel

In [ ]:
des.head()

In [ ]:
epsd.head()

### Now we are checking the porfile report the episode the table
- From the profile report we can get the whole summary of the data such as missing value,interation ,correlation etc

In [ ]:
from pandas_profiling import ProfileReport
prof = ProfileReport(epsd)
prof.to_file(output_file='Episode.html')

In [ ]:
prof

- We can clearly see the distribution,correlation missing etc for episode data set
- this will help us in understanding the overview of the dataset of we are going to work

### Necessary funtions

In [ ]:
# Genersting the pie chart
def pie_chart(df,col,path):
  label = df[col].value_counts().index.tolist()
  fig = plt.figure(figsize=(10,6))
  ax = (df[col].value_counts()*100.0 /len(df))\
  .plot.pie(startangle=90,autopct='%.1f%%', labels =label, fontsize=12)                                                                           
  ax.set_title('% '+str(col))
  # plt.savefig(path+str(col1)+'.png')
  plt.show()

# Relation between categorical variable
def categorical_summarized(path,dataframe, x=None, y=None, hue=None,palette='Set1', verbose=True):
    '''
    Helper function that gives a quick summary of a given column of categorical data
    Arguments
    =========
    dataframe: pandas dataframe
    x: str. horizontal axis to plot the labels of categorical data, y would be the count
    y: str. vertical axis to plot the labels of categorical data, x would be the count
    hue: str. if you want to compare it another variable (usually the target variable)
    palette: array-like. Colour of the plot
    Returns
    =======
    Quick Stats of the data and also the count plot
    '''
    if x == None:
        column_interested = y
    else:
        column_interested = x
    series = dataframe[column_interested]
    print(series.describe())
    print('mode: ', series.mode())
    if verbose:
        print('='*80)
        print(series.value_counts())

    sns.countplot(x=x, y=y, hue=hue, data=dataframe, palette=palette)
    plt.xlabel(str(x),Weight='bold',fontsize=12)
    plt.ylabel("Count",weight='bold',fontsize=12)
    if x== None:
        plt.title('Relation_between'+str(y)+'_'+'and'+'_'+str(hue))
#         g= plt.savefig(path+str(y)+'_'+str(hue)+'.png')
        labels = dataframe[column_interested].value_counts().index.tolist()
        labels.sort()
        labels=[textwrap.fill(text,10) for text in labels]
        pos = np.arange(len(labels)) 
        plt.yticks(pos, labels)
    else:
        plt.title('Relation_between'+'_'+str(x)+'_'+'and'+'_'+str(hue),weight='bold',fontsize=14)
#         g= plt.savefig(path+str(x)+'_'+str(hue)+'.png')
        labels = dataframe[column_interested].value_counts().index.tolist()
        labels.sort()
        labels=[textwrap.fill(text,10) for text in labels]
        pos = np.arange(len(labels)) 
        plt.xticks(pos, labels)
        plt.legend(loc='upper right')
    plt.show()

 # This helper fucntion help to find relation between categorical and numerical variable and visualize stack chart   
def stack_plot_sum(df,x_axis,y_axis,hue,title,path):
  t1 = df.groupby([x_axis,hue])[[y_axis]].count().add_prefix('sum_of_').reset_index()
  t2 = t1.pivot(x_axis,hue,'sum_of_'+y_axis)
  ax = t2.plot(kind='bar',stacked=True)
  plt.xticks(rotation=90)
#   ax.legend(["Not_Fraud", "Fraud"])
  plt.xlabel(str(x_axis),fontweight ="bold",fontsize=14)
  plt.ylabel('Frequency',fontweight ="bold",fontsize=14)
  plt.title(title,fontweight ="bold",fontsize=16)
  # plt.savefig(path+title+'.png')
  plt.show()

    
# Relation between the categorical variable    
def rel_cat(df,x_axis,y_axis,path,stacked=None):
    temp =pd.crosstab(df[x_axis],df[y_axis])
    temp.plot(kind='bar',stacked=stacked,grid=False)
    plt.xlabel(str(x_axis),weight='bold',fontsize=12)
    plt.ylabel(str(y_axis),weight='bold',fontsize=12)
    plt.title(str(x_axis)+'_'+'and'+'_'+str(y_axis),weight='bold',fontsize=14)
    plt.xticks(rotation=0,fontsize=12)
    plt.yticks(fontsize=12)
    labels = df[x_axis].value_counts().index.tolist()
    labels.sort()
    labels=[textwrap.fill(text,10) for text in labels]
    pos = np.arange(len(labels)) 
    plt.xticks(pos, labels)
#     plt.legend()
#     plt.savefig(path+str(x_axis)+'_'+'and'+'_'+str(y_axis)+'.jpg')    
    plt.show()
    

# Helper fucntion helps to find the relation between the categorical and numerical variable
def rel_num_cat(df,x_axis,y_axis,palette,path):
    t =df.groupby([x_axis])[y_axis].sum().reset_index()
    heroes = t.sort_values(by=y_axis,ascending=False)[:20].reset_index(drop=True)
#     heroes

    fig,ax = plt.subplots(figsize=(18,6))
    g= sns.barplot(x=heroes['heroes'],y=heroes[y_axis],data=heroes,ax=ax,palette=palette)
    plt.xlabel(str(x_axis),weight='bold',fontsize=12)
    plt.ylabel(str(y_axis),weight='bold',fontsize=12)
    plt.title('Number of '+str(y_axis)+'_'+'for_each'+'_'+str(x_axis),weight='bold',fontsize=14)
    plt.xticks(rotation=0)
    for index, row in heroes.iterrows():
        g.text(row.name,row[1], round(row[1]), color='black', ha="center")
    labels = heroes[x_axis].tolist()
    # labels.sort()
    labels=[textwrap.fill(text,10) for text in labels]
    pos = np.arange(len(labels)) 
    plt.xticks(pos, labels)
#     plt.savefig('Number of '+str(y_axis)+ 'for each'+ str(x_axis)+'.jpg')
    plt.show()

In [ ]:
epsd.head()

In [ ]:
epsd.info()

##### We can seet the datastype of each variable

### Distribution of the Heroes based on Gender

In [ ]:
pie_chart(epsd,'heroes_gender',8)

*Here male heroes are much more the female ML heroes*

### Distribution of category 

In [ ]:
pie_chart(epsd,'category',8)

> Here Kaggle and Industry covers more than 75% of the category

### Distribuition of the Flavour tea used in the eahc Episodes

In [ ]:
pie_chart(epsd,'flavour_of_tea',8)

> Ginger Chai,Masala chai,Sulemani Chai,Herbal chai and Kesar Rose chai are mostly used in the Episodes> 

In [ ]:
# sns.set_style("white")
sns.countplot(y=epsd['heroes_nationality'],data=epsd,palette='winter')
plt.xlabel('Count',weight='bold',fontsize=12)
plt.ylabel('Heros_Nationality',weight='bold',fontsize=12)
plt.title('Count of Heros from different Nationality',weight='bold',fontsize=14)
plt.show()

#### Most ML Heroes are from India,USA,Canada,Germany,Russia and France

In [ ]:
f, ax = plt.subplots(figsize=(10,6))
sns.countplot(y=epsd['recording_time'],data=epsd,palette='winter_r',ax=ax)
plt.xlabel('Count',weight='bold',fontsize=12)
plt.ylabel('Recording_time',weight='bold',fontsize=12)
plt.title('Count of recoding time',weight='bold',fontsize=14)
for p in ax.patches:
    width = p.get_width()
    ax.text(width+1.,
            p.get_y()+p.get_height()/3. + 0.2,
            '{:1.2f}'.format(width),
            ha="center")
plt.show()

### Most of the Recordings the happen at Night

In [ ]:
c_palette = ['tab:blue', 'tab:orange','tab:green','tab:red']
categorical_summarized(8,epsd,x='category',hue='heroes_gender',palette='winter')

It is showing that CTDS  dosen't have female Kaggler and it having very less count of female in other category

In [ ]:
rel_cat(epsd,'flavour_of_tea','recording_time',8,stacked=True)

Kesar Rose Chai are mostly have in the night and on the other hand, masala tea are have during the morning

In [ ]:
epsd.columns

In [ ]:
epsd['category'].value_counts()

In [ ]:
plt.rcParams['figure.figsize']=18,6
rel_cat(epsd,'heroes_nationality','category',8,stacked=True)

In [ ]:
plt.rcParams['figure.figsize']=18,6
categorical_summarized(8,epsd,x='flavour_of_tea',hue='heroes_gender',palette='winter')

### Ginger tea , Apple Cinnamon, Masala Chai, Paan Rose Green Tea are mostly have by Males Heroes

In [ ]:
categorical_summarized(8,epsd,x='flavour_of_tea',hue='category',palette='Blues_r')

### Ginger tea are mostly have by the Kaggler

In [ ]:
t =epsd.groupby(['heroes'])['youtube_subscribers'].sum().reset_index()
heroes = t.sort_values(by='youtube_subscribers',ascending=False)[:10].reset_index(drop=True)
heroes

In [ ]:
t =epsd.groupby(['heroes'])['youtube_subscribers'].sum().reset_index()
heroes = t.sort_values(by='youtube_subscribers',ascending=False)[:20].reset_index(drop=True)
heroes

fig,ax = plt.subplots(figsize=(18,6))
g= sns.barplot(x=heroes['heroes'],y=heroes['youtube_subscribers'],data=heroes,ax=ax,palette='Blues_r')
plt.xlabel('Heros',weight='bold',fontsize=12)
plt.ylabel('youtube_subscribers',weight='bold',fontsize=12)
plt.title('Number of Subscribers for each Heroes',weight='bold',fontsize=14)
plt.xticks(rotation=0)
for index, row in heroes.iterrows():
    g.text(row.name,row.youtube_subscribers, round(row.youtube_subscribers), color='black', ha="center")
labels = heroes['heroes'].tolist()
# labels.sort()
labels=[textwrap.fill(text,10) for text in labels]
pos = np.arange(len(labels)) 
plt.xticks(pos, labels)
plt.show()

> Showing the top 20 kaggler having subscribers and 
> Jeremy Howard having most number of subscribers than other


In [ ]:
plt.rcParams['figure.figsize']=10,6
rel_cat(epsd,'recording_time','category',8,stacked=True)

> Industry having most of ther recording at night where as the Kagglers having recording at afternoon, Evening, Morning and Night

In [ ]:
plt.rcParams['figure.figsize']=18,6
categorical_summarized(8,epsd,x='recording_time',hue='category',palette='BuPu_r')

In [ ]:
x = epsd['episode_id']
y= epsd['episode_duration']

# Color palette
blue, = sns.color_palette("muted", 1)

# Make the plot
fig,ax = plt.subplots(figsize=(24,8))
ax.plot(x, y, color=blue, lw=3)
ax.fill_between(x, 0, y, alpha=.3)
ax.set(xlim=(0, len(x)-1), ylim=(0, None), xticks=x)
plt.show()


In [ ]:
rel_num_cat(epsd,'heroes','spotify_starts','winter',8)

In [ ]:
rel_num_cat(epsd,'heroes','anchor_plays','brg',8)

In [ ]:
rel_num_cat(epsd,'heroes','youtube_likes','hsv',8)

In [ ]:
rel_num_cat(epsd,'heroes','youtube_watch_hours','gnuplot',8)

In [ ]:
data_epd = epsd
data_epd['avg_duration']=  data_epd['youtube_watch_hours']*60/ data_epd['youtube_views']
rel_num_cat(data_epd,'heroes','avg_duration','twilight_shifted',8)

In [ ]:
epsd.columns

In [ ]:
rel_num_cat(epsd,'heroes','youtube_nonimpression_views','Blues_r',8)
# Jeremy Howard and Parul having  youtube non impression views

In [ ]:
rel_num_cat(epsd,'heroes','youtube_impression_views','winter',8)

In [ ]:
rel_num_cat(epsd,'heroes','youtube_dislikes','Spectral',8)

In [ ]:
rel_num_cat(epsd,'heroes','youtube_subscribers','Spectral',8)

In [ ]:
t1 = epsd.groupby(['episode_name','category'])[['youtube_ctr']].sum().add_prefix('sum_of_').reset_index()
t1.head()

In [ ]:
plt.figure(figsize=(24,6))
sns.lineplot(x='episode_name',y='sum_of_youtube_ctr',hue='category',data=t1)
plt.xticks(rotation=90)
plt.show()

# Episode wise category

In [ ]:
epsd.head()

In [ ]:
epsd['spotify_listeners'].plot()
epsd['apple_listeners'].plot()

## Lets Work on description dataset so that we can get some information 

In [ ]:
des.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/ import string
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import nltk
import random
import string

In [ ]:

def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text


def text_preprocessing(text):
    """
    Cleaning and parsing the text.

    """
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    nopunc = clean_text(text)
    tokenized_text = tokenizer.tokenize(nopunc)
    #remove_stopwords = [w for w in tokenized_text if w not in stopwords.words('english')]
    combined_text = ' '.join(tokenized_text)
    return combined_text


In [ ]:
# Applying the cleaning function to both Descriptions datasets
des['text_clean'] = des['description'].apply(str).apply(lambda x: text_preprocessing(x))

In [ ]:
# Analyzing Text statistics

des['text_len'] = des['text_clean'].astype(str).apply(len)
des['text_word_count'] = des['text_clean'].apply(lambda x: len(str(x).split()))

In [ ]:
plt.rcParams['figure.figsize']=10,6
sns.distplot(des['text_len'],color='red')
plt.xlabel('text_len',weight='bold',fontsize=12)
plt.ylabel('Count',weight='bold',fontsize=12)
plt.title('Distribution of description',weight='bold',fontsize=14)
plt.show()

In [ ]:
#source of code : https://medium.com/@cristhianboujon/how-to-list-the-most-common-words-from-text-corpus-using-scikit-learn-dad4d0cab41d
def get_top_n_words(corpus, n=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    """
    vec = CountVectorizer(stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
#Distribution of top unigrams
des_unigrams = get_top_n_words(des['text_clean'],20)

df1 = pd.DataFrame(des_unigrams, columns = ['Text' , 'count'])
df1.groupby('Text').sum()['count'].sort_values(ascending=True).plot(kind='barh',color='g')
plt.ylabel('words',weight='bold',fontsize=12)
plt.xlabel('Counts',weight='bold',fontsize=12)
plt.title('Top 20 unigrams in description text',weight='bold',fontsize=14)
plt.show()

In [ ]:
def get_top_n_gram(corpus,ngram_range,n=None):
    vec = CountVectorizer(ngram_range=ngram_range,stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
#Distribution of top Bigrams
des_bigrams = get_top_n_gram(des['text_clean'],(2,2),20)

df1 = pd.DataFrame(des_bigrams, columns = ['Text' , 'count'])
df1.groupby('Text').sum()['count'].sort_values(ascending=True).plot(kind='barh',color='r',alpha=0.5)
plt.ylabel('words',weight='bold',fontsize=12)
plt.xlabel('Counts',weight='bold',fontsize=12)
plt.title('Top 20 bigrams in description text',weight='bold',fontsize=14)
plt.show()

In [ ]:
#Distribution of top Trigrams
des_trigrams = get_top_n_gram(des['text_clean'],(3,3),20)

df1 = pd.DataFrame(des_trigrams, columns = ['Text' , 'count'])
df1.groupby('Text').sum()['count'].sort_values(ascending=True).plot(kind='barh',color='b',alpha=0.5)
plt.ylabel('words',weight='bold',fontsize=12)
plt.xlabel('Counts',weight='bold',fontsize=12)
plt.title('Top 20 trigrams in description text',weight='bold',fontsize=14)
plt.show()

In [ ]:
#source of code : https://medium.com/@cristhianboujon/how-to-list-the-most-common-words-from-text-corpus-using-scikit-learn-dad4d0cab41d
def get_top_n_words(corpus, n=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    """
    vec = CountVectorizer(stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
top_words_in_description_text = get_top_n_words(des['text_clean'])

p1 = [x[0] for x in top_words_in_description_text[:20]]
p2 = [x[1] for x in top_words_in_description_text[:20]]

In [ ]:
# Top Descriptions word
sns.barplot(x=p2,y=p1,palette='winter')
# plt.xticks(rotation=45)
plt.yticks(fontsize=12)
plt.ylabel('Words',weight='bold',fontsize=12)
plt.xlabel('Counts',weight='bold',fontsize=12)
plt.title('Top 20 description Words',weight='bold',fontsize=14)
plt.show()

In [ ]:
from wordcloud import WordCloud
fig,ax = plt.subplots(figsize=[40,30])
wordcloud1 = WordCloud( background_color='white',
                        width=600,
                        height=400).generate(" ".join(des['text_clean']))
ax.imshow(wordcloud1)
ax.axis('off')
ax.set_title('Descriptions text',fontsize=20);

Reference: https://www.kaggle.com/parulpandey/how-to-explore-the-ctds-show-data,
           https://www.kaggle.com/vpkprasanna/insights-on-chai-time-data-science